In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
SHAPE = None
NREALS = 500
NFREQS = 40
NLOUDEST = 10
BGL = 1

TOL=0.01
MAXBADS=5

NVARS = 21

NPSRS = 40
NSKIES = 100

ALL_TARGETS = ['gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_time',  'hard_gamma_inner']

# Plot Number of Sources Contributing to EV_SS

In [ ]:
BUILD_ARRAYS=False

ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', yscale='linear', height=7)
fig.text(0.08, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, y1, y2, _, _, _, _ = detstats.build_noise_arrays(target)
    else:
        _, y1, y2, _, _, _, _ = detstats.get_noise_arrays_temp(target)

    y1 = y1.reshape(NVARS, NSKIES*NREALS)
    y2 = y2.reshape(NVARS, NREALS*NSKIES)

    h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
    h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')

plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['$0.50$', '$0.01$']
fig.legend(handles=[h1, h2], labels=labels, bbox_to_anchor=(0.5,-0.02), loc='lower center',
           ncols=2, title='$x$ threshold')

### with n_excl=3: hard_time

In [ ]:
# load data

nexcl=3
target='hard_time'
white_noise, y1, y2, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target, nexcl=nexcl, save_temp=False)


In [ ]:
# plot data 
ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, ax = plot.figax_single(
                              xscale='linear', yscale='linear',)
ax.set_ylabel(ylabel)
ax.set_title(title, fontsize=12)
ax.set_xlabel(plot.PARAM_KEYS[target])

xx, _ = detstats.get_ratio_arrays(target)

y1 = y1.reshape(NVARS, NSKIES*NREALS)
y2 = y2.reshape(NVARS, NREALS*NSKIES)

h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')


labels = ['$0.50$', '$0.01$']

ax.legend(handles=[h1, h2], labels=labels, loc='upper left',
           ncols=2, title='$x$ threshold')


############################# 2nd plot #########################

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0
fobs, _ = holo.utils.pta_freqs()

fig2, ax2 = plot.figax_single(xscale='linear')
ax2.set_ylabel(ylabel)
ax2.set_title(title, fontsize=12)
psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                        reshape_freqs=False) # V,F,R,L

yy_wn = white_noise.reshape(NVARS, NREALS)
yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

h1 = plot.draw_med_conf_color(ax2, xx, yy_wn, color='grey', linestyle=':')
h2 = plot.draw_med_conf_color(ax2, xx, yy_bg, color='tab:blue', linestyle='--')
h3 = plot.draw_med_conf_color(ax2, xx, yy_ss, color='black', fracs=[0.5,])
h4 = plot.draw_med_conf_color(ax2, xx, yy_l1, color='#00ff00')
h5 = plot.draw_med_conf_color(ax2, xx, yy_l2, color='#ff0000' )
h6 = plot.draw_med_conf_color(ax2, xx, yy_lt, color='#4000ff')
labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
ax2.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=3, columnspacing=1.5)

### with n_excl=3: gsmf_phi0

In [ ]:
# load data

nexcl=3
target='gsmf_phi0'
white_noise, y1, y2, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target, nexcl=nexcl, save_temp=False)


In [ ]:
# plot data 
ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, ax = plot.figax_single(
                              xscale='linear', yscale='linear',)
ax.set_ylabel(ylabel)
ax.set_title(title, fontsize=12)
ax.set_xlabel(plot.PARAM_KEYS[target])

xx, _ = detstats.get_ratio_arrays(target)

y1 = y1.reshape(NVARS, NSKIES*NREALS)
y2 = y2.reshape(NVARS, NREALS*NSKIES)

h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')


labels = ['$0.50$', '$0.01$']

ax.legend(handles=[h1, h2], labels=labels, loc='upper left',
           ncols=2, title='$x$ threshold')


############################# 2nd plot #########################

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0
fobs, _ = holo.utils.pta_freqs()

fig2, ax2 = plot.figax_single(xscale='linear')
ax2.set_ylabel(ylabel)
ax2.set_title(title, fontsize=12)
psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                        reshape_freqs=False) # V,F,R,L

yy_wn = white_noise.reshape(NVARS, NREALS)
yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

h1 = plot.draw_med_conf_color(ax2, xx, yy_wn, color='grey', linestyle=':')
h2 = plot.draw_med_conf_color(ax2, xx, yy_bg, color='tab:blue', linestyle='--')
h3 = plot.draw_med_conf_color(ax2, xx, yy_ss, color='black', fracs=[0.5,])
h4 = plot.draw_med_conf_color(ax2, xx, yy_l1, color='#00ff00')
h5 = plot.draw_med_conf_color(ax2, xx, yy_l2, color='#ff0000' )
h6 = plot.draw_med_conf_color(ax2, xx, yy_lt, color='#4000ff')
labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
ax2.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=3, columnspacing=1.5)

# Plot Noise

In [ ]:
fobs, _ = holo.utils.pta_freqs()

In [ ]:
BUILD_ARRAYS=False

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.get_noise_arrays_temp(target)

    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
    yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
    yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

    h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1, color='#00ff00')
    h5 = plot.draw_med_conf_color(ax, xx, yy_l2, color='#ff0000' )
    h6 = plot.draw_med_conf_color(ax, xx, yy_lt, color='#4000ff')



plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
fig.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.get_noise_arrays_temp(target)

    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
    yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
    yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

    h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1, color='#00ff00')
    h5 = plot.draw_med_conf_color(ax, xx, yy_l2, color='#ff0000' )
    h6 = plot.draw_med_conf_color(ax, xx, yy_lt, color='#4000ff')



plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
fig.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='Ratio of Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Ratio of Power Spectral Density [Hz$^{-3}$/Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.get_noise_arrays_temp(target)

    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1l2 = (psd_ss[:,ff,:,0]/psd_ss[:,ff,:,1]).reshape(NVARS, NREALS)
    yy_l1lt = (psd_ss[:,ff,:,0]/psd_ss[:,ff,:,9]).reshape(NVARS, NREALS)

    # h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    # h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    # h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h5 = plot.draw_med_conf_color(ax, xx, yy_l1lt, color='tab:blue', )
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1l2, color='#4000ff', )
    


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['1st/2nd L', '1st/10th L']
fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)

# Look at rank of loudest vs rank of DP

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )
hc_ss = data[10]['hc_ss']
gamma_ssi = dsdat[10]['gamma_ssi']

In [ ]:
print(gamma_ssi.shape)

In [ ]:
ff=0 # just look at lowest frequency bin
argmax = np.argmax(gamma_ssi[ff], axis=(-1))
argmax += 1
print(f"{argmax.shape=}, {np.max(argmax)=} should = 10")
fig, ax = plot.figax(xlabel='Rank of Loudest', ylabel='Number of Most Detectable',
                     xscale='linear')
bins = np.arange(0,11, dtype=float)
print(bins)
bins += 0.5

ax.hist(np.swapaxes(argmax,0,1), bins)
ax.set_xticks([1,2,3,4,5,6,7,8,9,10])

In [ ]:
ff=0 # just look at lowest frequency bin
argmax = np.argmax(gamma_ssi[ff], axis=(-1))
argmax += 1
print(f"{argmax.shape=}, {np.max(argmax)=} should = 10")
fig, ax = plot.figax(xlabel='Rank of Loudest', ylabel='Number of Most Detectable',
                     xscale='linear')
bins = np.arange(0,11, dtype=float)
print(bins)
bins += 0.5

ax.hist(argmax.flatten(), bins, density=True,)
ax.set_xticks([1,2,3,4,5,6,7,8,9,10])

# Look at dp_ssi comparison

In [ ]:
BUILD_ARRAYS=False

title='Ratio of detection probabilities'
ylabel = 'DP_1st/DP_2nd'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)

   
    yy = (dp_l1/dp_l2).reshape(NVARS, NREALS*NSKIES)

    h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
  


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='Ratio of detection probabilities'
ylabel = 'DP_1st/DP_2nd'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel('DP_1st')

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)

   
    # sort = np.argsort(dp_l1.reshape(NVARS, NREALS*NSKIES))
    # print(sort.shape)
    # yy = (dp_l1/dp_l2).reshape(NVARS, NREALS*NSKIES)
    # print(yy.shape)
    # yy=yy[sort]
    # xx = dp_l1[sort]

    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    # ax.hist2d(dp_l1.flatten(), (dp_l1/dp_l2).flatten(), bins=10)
    ax.scatter(dp_l1.flatten(), (dp_l1/dp_l2).flatten(), alpha=0.1, s=1)


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='2d histogram, log(N)'
ylabel = 'log(DP_1st/DP_2nd)'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              height=7, xscale='linear', yscale='linear')
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel('log(DP_1st)')

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)

   
    # sort = np.argsort(dp_l1.reshape(NVARS, NREALS*NSKIES))
    # print(sort.shape)
    # yy = (dp_l1/dp_l2).reshape(NVARS, NREALS*NSKIES)
    # print(yy.shape)
    # yy=yy[sort]
    # xx = dp_l1[sort]

    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    hist, dprat_edges, dp1st_edges,  = np.histogram2d(np.log10(dp_l1/dp_l2).flatten(), np.log10(dp_l1).flatten(), bins=60)
    dp1st_grid, dprat_grid  = np.meshgrid(dp1st_edges, dprat_edges)
    im = ax.pcolormesh(dp1st_grid, dprat_grid, np.log10(hist))


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='2d histogram, log(N)'
ylabel = 'log(DP_2nd)'
xlabel = 'log(DP_1st)'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              height=7, xscale='linear', yscale='linear')
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.text(0.5, 0.08, xlabel, ha='center', va='center')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)



    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    dp2nd_edges = np.linspace(np.log10(np.min(dp_l2)), 
                               np.log10(np.max(dp_l2)), 100)
    dp1st_edges = np.linspace(np.log10(np.min(dp_l1)), 
                               np.log10(np.max(dp_l1)), 200)
    hist, dp2e, dp1e,  = np.histogram2d(np.log10(dp_l2).flatten(), np.log10(dp_l1).flatten(), 
                                                      bins=(dp2nd_edges, dp1st_edges))
    dp1st_grid, dp2nd_grid  = np.meshgrid(dp1st_edges, dp2nd_edges)
    im = ax.pcolormesh(dp1st_grid, dp2nd_grid, np.log10(hist))
    cbar = plt.colorbar(im ,ax=ax, )
    ax.set_xlim(-1, 0)
    ax.set_ylim(-1,0)


plt.subplots_adjust(wspace=0.05, hspace=0.2, top=0.93)
# plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.4, 
#              orientation='horizontal', pad=0.22, label='$\log N$', )



# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
array = np.random.random(4*50*10*3).reshape(4,50,10,3)
maxes = np.max(array, axis=(0,3))
print(maxes.shape)
array[array==maxes[np.newaxis,:,:,np.newaxis]]=0
print(array.shape)
second = np.max(array, axis=(0,3))
print(second.shape)


# Check EV_SS logged

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )

In [ ]:
logevss = []
for ii in range(21):
    logevss.append(np.log10(dsdat[ii]['ev_ss']))
logevss = np.array(logevss)
print(logevss.shape)

In [ ]:
fig, ax = plot.figax(xscale='linear', yscale='linear',
                     xlabel=plot.PARAM_KEYS[target],
                     ylabel='$\log{\langle N_\mathrm{SS} \\rangle}$')
yy = logevss.reshape(21, 500*100)
for pp in [50, 95]:
    percs = [50-pp/2, 50+pp/2]
    med, *conf = np.percentile(yy, (50, *percs), axis=1)
    ax.plot(xx, med)
    ax.fill_between(xx, *conf, alpha=0.15)